In [1]:
from Datasets.Paris.load_data import data_loader
from utils.cluster import KMEANS
from utils.get_features import SIFT
from utils.find_similar_vectors import KNN
import matplotlib.pyplot as plt
import numpy as np

In [2]:
images, image_labels = data_loader()

100%|██████████| 12/12 [01:48<00:00,  9.05s/it]


In [3]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(
    images, image_labels, shuffle=True, test_size=0.1)

In [4]:
from importlib import reload

from utils import get_features
reload(get_features)
from utils.get_features import SIFT

In [5]:
orb_detector = SIFT('sift')
features = orb_detector.get_features(train_images)

features_array = np.concatenate(features, axis=0)

100%|██████████| 5752/5752 [14:49<00:00,  6.47it/s]


In [6]:
num_features, dim = features_array.shape
num_clusters = 500

kmeans = KMEANS(dim, num_clusters=num_clusters, niter=500)
kmeans.train(features_array)

Sampling a subset of 128000 / 19698092 for training
Clustering 128000 points in 128D to 500 clusters, redo 1 times, 500 iterations
  Preprocessing in 3.01 s
  Iteration 499 (32.38 s, search 28.97 s): objective=8.70966e+09 imbalance=1.113 nsplit=0       


In [7]:
from tqdm import tqdm


hists = []

for f in tqdm(features):
    labels = kmeans.search(f)[:, 0]
    counts = np.unique(labels, return_counts=True)
    arr = np.zeros(num_clusters)
    arr[counts[0]] = counts[1]
    arr /= np.linalg.norm(arr)
    hists.append(arr[None, :]) # dim: 1 x num_clusters

hists = np.concatenate(hists, axis=0)    

100%|██████████| 5752/5752 [00:30<00:00, 186.80it/s]


In [8]:
knn = KNN(num_clusters)
knn.fit(hists)
knn.findKNearest(hists[:3])

array([[   0, 5372, 3431, 2324, 3518, 1248,  814, 4142, 2744, 5587],
       [   1, 4927, 3077, 1553,  107, 5359, 3010,  931, 2469, 1033],
       [   2, 3467, 4391, 4883,  610, 4688, 3039, 5029, 2038,  259]])

# Testing

In [9]:
def K_precisions(true_labels, ret_labels):
    eq = ret_labels == true_labels
    cumulative = np.cumsum(eq)
    precisions = cumulative / (np.arange(len(ret_labels)) + 1)
    return precisions

In [10]:
K = 5
precision = np.zeros(K)

APs = []
for image, true_label in tqdm(zip(test_images, test_labels)):
    # get the orb features
    f = orb_detector.__get_features__(image)
    # it is possible that no descriptors are obtained
    if f is None:
        print(f'{true_label}: no features extracted')
        continue

    f = np.array(f)
    
    # histogram
    labels = kmeans.search(f)[:, 0]
    counts = np.unique(labels, return_counts=True)
    arr = np.zeros(num_clusters)
    arr[counts[0]] = counts[1]
    arr /= np.linalg.norm(arr)
    
    # finding the nearest neighbours
    indices = knn.findKNearest(arr[None, :], K)[0]
    # finding the labels associated with retrieved
    # neighbours
    ret_labels = np.array([train_labels[i] for i in indices])
    APs.append(K_precisions(true_label, ret_labels))


APs = np.array(APs)
mAP = np.mean(APs)    

640it [02:45,  3.86it/s]


In [19]:
non_general_indices = np.where(np.array(test_labels) != 'general')
APs[non_general_indices].mean(axis=0)

array([0.55693582, 0.52173913, 0.49689441, 0.48033126, 0.47039337])

In [11]:
APs.mean(axis=0)

array([0.5296875 , 0.4921875 , 0.47395833, 0.46484375, 0.455     ])

In [12]:
mAP

0.48313541666666665